In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.3e6
length = 50

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
action_store = []
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
aa = [1] * 5
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    action_store.append(action)
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, score:-229.27
 Observe Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
 True State Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2300000.00, score:-554.35
 Observe Time: 1 | S: 25495797.0, E: 143.0, A: 0.0, I: 47.3, D: 0.0, R: 0.0
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
 True State Time: 2 | S: 25495676.0, E: 199.6, A: 0.0, I: 112.6, D: 0.1, R: 11.8, Budget: 2300000.00, score:-1016.39
 Observe Time: 2 | S: 25495678.0, E: 199.0, A: 0.0, I: 112.6, D: 0.1, R: 0.0
 Time: 2 | new_e:104.9, cum_e:232.1, new_i:91.3, cum_i:204.0
 True State Time: 3 | S: 25495501.2, E: 274.0, A: 0.0, I: 203.8, D: 0.2, R: 20.8, Budget: 2300000.00, score:-1674.21
 Observe Time: 3 | S: 25495510.0, E: 273.0, A: 0.0, I: 203.8, D: 0.2, R: 12.0
 Time: 3 | new_e:148.7, cum_e:380.8, new_i:128.5, cum_

 True State Time: 21 | S: 24576604.4, E: 8.7, A: 260.3, I: 823.4, D: 4.8, R: 918298.5, Budget: 1643688.16, score:-10497.28
 Observe Time: 21 | S: 24576772.0, E: 0.0, A: 260.3, I: 823.4, D: 4.8, R: 918291.0
 Time: 21 | new_e:18.5, cum_e:2354.0, new_i:41.2, cum_i:3127.7
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
pyomo fail
 True State Time: 22 | S: 24527840.8, E: 23.6, A: 222.9, I: 769.2, D: 4.9, R: 967138.5, Budget: 1598806.43, score:-10549.12
 Observe Time: 22 | S: 24528014.0, E: 16.0, A: 222.9, I: 769.2, D: 4.9, R: 967131.0
 Time: 22 | new_e:10.7, cum_e:2364.6, new_i:33.5, cum_i:3161.2
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 23 | S: 24486449.2, E: 12.3, A: 207.2, I: 706.6, D:

 True State Time: 44 | S: 23553913.5, E: 16.1, A: 36.4, I: 141.0, D: 6.0, R: 1941887.0, Budget: 983408.13, score:-11347.59
 Observe Time: 44 | S: 23554296.0, E: 0.0, A: 36.4, I: 141.0, D: 6.0, R: 1941881.0
 Time: 44 | new_e:7.0, cum_e:2534.8, new_i:10.3, cum_i:3512.3
 True State Time: 45 | S: 23517821.3, E: 16.4, A: 35.1, I: 133.6, D: 6.1, R: 1977987.6, Budget: 959871.67, score:-11379.18
 Observe Time: 45 | S: 23518215.0, E: 0.0, A: 35.1, I: 133.6, D: 6.1, R: 1977980.0
 Time: 45 | new_e:7.1, cum_e:2541.9, new_i:10.2, cum_i:3522.5
 True State Time: 46 | S: 23478767.6, E: 17.2, A: 33.5, I: 127.1, D: 6.1, R: 2017048.5, Budget: 936402.36, score:-11412.15
 Observe Time: 46 | S: 23479171.0, E: 0.0, A: 33.5, I: 127.1, D: 6.1, R: 2017042.0
 Time: 46 | new_e:7.4, cum_e:2549.3, new_i:10.4, cum_i:3532.8
 True State Time: 47 | S: 23446765.6, E: 17.2, A: 33.1, I: 121.5, D: 6.1, R: 2049056.4, Budget: 912971.90, score:-11445.18
 Observe Time: 47 | S: 23447181.0, E: 0.0, A: 33.1, I: 121.5, D: 6.1, R: 

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 65 | S: 22811039.9, E: 93.0, A: 52.1, I: 193.4, D: 6.5, R: 2684615.1, Budget: 533312.29, score:-12881.90
 Observe Time: 65 | S: 22811874.0, E: 1.0, A: 52.1, I: 193.4, D: 6.5, R: 2684601.0
 Time: 65 | new_e:38.6, cum_e:2882.9, new_i:41.3, cum_i:3911.3
 True State Time: 66 | S: 22794568.6, E: 101.7, A: 59.9, I: 210.5, D: 6.5, R: 2701052.8, Budget: 518650.76, score:-13066.11
 Observe Time: 66 | S: 22795467.0, E: 0.0, A: 59.9, I: 210.5, D: 6.5, R: 2701035.0
 Time: 66 | new_e:42.2, cum_e:2925.1, new_i:45.4, cum_i:3956.8
pyomo fail
 True State Time: 67 | S: 22770230.3, E: 117.7, A: 61.4, I: 229.6, D: 6.5, R: 2725354.5, Budget: 505765.82, score:-13278.93
 Observe Time: 67 | S: 22771198.0, E: 0.0, A: 61.4, I: 229.6, D: 6.5, R: 2725337.0
 Time: 67 | new_e:48.7, cum_e:2973.8, new_i:51.7, cum

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 82 | S: 22563878.3, E: 1281.6, A: 345.2, I: 1824.9, D: 8.2, R: 2928661.7, Budget: 307025.42, score:-27522.12
 Observe Time: 82 | S: 22569269.0, E: 46.0, A: 345.2, I: 1824.9, D: 8.2, R: 2928411.0
 Time: 82 | new_e:539.3, cum_e:6232.8, new_i:533.8, cum_i:7263.5
    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.14.6\x3a Maximum Number of Iterations
      

In [10]:
result_state = pd.DataFrame(state_buffer)
result_state.to_csv(r'MPC_low_state_16.csv')
result_new = pd.DataFrame(new_buffer)
result_new.to_csv(r'MPC_low_new_16.csv')

In [11]:
import pickle
pickle.dump(action_store, open("MPC_low_action.pth", 'wb'))

In [8]:
(time.time() - tic) / ( T - start_time + 1)

161.64567482050728

In [9]:
6959.9 + 8940.0

15899.9